In [1]:
import numpy as np
import pandas as pd 
import networkx as nx
import math
import sympy as sp
import matplotlib.pyplot as plt
import time

from sympy import *
from numpy import linalg as LA
from numpy.linalg import matrix_rank

In [2]:
#Establecer directorio de datos

DIR='/Users/alex/Desktop'
FILE='/MatrizAdjSano/MatrizAdjSano/adjmatrix_UNS.txt'

file='{}{}'.format(DIR,FILE)

print('File directory:{}'.format(file))



File directory:/Users/alex/Desktop/MatrizAdjSano/MatrizAdjSano/adjmatrix_UNS.txt


In [3]:
data=pd.read_table(file)

In [4]:
data.head()

,ENSG00000000003,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000001036,ENSG00000001167,ENSG00000001460,ENSG00000001461,ENSG00000001497,...,hsa-mir-941-5,hsa-mir-942,hsa-mir-943,hsa-mir-944,hsa-mir-95,hsa-mir-9500,hsa-mir-96,hsa-mir-98,hsa-mir-99a,hsa-mir-99b
0,1.0,0.103197,0.152306,0.107439,0.151863,0.050207,0.047916,0.152306,0.162813,0.032128,...,0.038779,0.016197,0.052990,0.062391,0.070298,0.038779,0.044215,0.259474,0.151107,0.074188
1,NaN,1.000000,0.120830,0.128810,0.121863,0.076382,0.064962,0.120830,0.250164,0.102605,...,0.108833,0.124237,0.120423,0.125844,0.020272,0.108833,0.127723,0.069236,0.199134,0.112418
2,NaN,NaN,1.000000,0.051569,0.107554,0.057486,0.038834,0.334130,0.248721,0.048574,...,0.059112,0.044184,0.051950,0.091854,0.071886,0.059112,0.030633,0.097257,0.045373,0.075525
3,NaN,NaN,NaN,1.000000,0.032182,0.069682,0.042741,0.051569,0.126349,0.051244,...,0.054916,0.133838,0.062266,0.132682,0.021764,0.054916,0.103477,0.123546,0.184396,0.057522
4,NaN,NaN,NaN,NaN,1.000000,0.214658,0.029944,0.107554,0.108194,0.094873,...,0.102947,0.156264,0.069491,0.128388,0.018218,0.102947,0.094818,0.178692,0.051804,0.064486


In [5]:
a=np.diag(np.ones(16748),0)
A=data-a

In [6]:
threshold = A.mean().mean()

In [7]:
#https://stackoverflow.com/questions/13914920/networkx-extract-the-connected-component-containing-a-given-node-directed-grap?fbclid=IwAR0CLa62zO8y43PVzLDO54AZEo8Gozv4ai8hJDFKXXcgc8mZlN95wu3Xlio

#Función para ver qué forma tiene la matriz del data set que estamos
#estudiando 

def mat_shape(mat,n):
    
    new_data=mat.iloc[:n,:n]
    
    new_data2=new_data.apply(lambda x:(np.where(x<threshold,0,x)))
    adj_mat=new_data2.apply(lambda x:(np.where(x>threshold,1,x)))
    
    #completar la matriz triangular superior para tener la forma completa de una matriz
    # simétrica
    
    adj_mod=np.matrix(adj_mat.fillna(0))
    adj_mod_trans=adj_mod.transpose()
    simm_mat=adj_mod+adj_mod_trans
    
    
    return simm_mat


In [8]:
#Función para calcular el polinomio característico de la matriz laplaciana sin signo

def pol_char_from_signless_lap(mat,n):
    
    
    simm_mat=mat_shape(mat,n)
    
    Adj_to_nx=nx.from_numpy_matrix(simm_mat)
    L_from_Adj=nx.laplacian_matrix(Adj_to_nx)
    
    L_arr=L_from_Adj.toarray()
    L_np=np.matrix(L_arr)
    
    deg_Mat=L_np+simm_mat
    
    L_WO_Sign=deg_Mat+simm_mat
    
    pol_char=np.poly(L_WO_Sign)
    
    
    
    return pol_char

In [9]:
pol_char_from_signless_lap(A,3)

array([ 1., -6.,  9., -4.])

In [10]:
#Función para obtener el polinomio característico de la matriz laplaciana sin signo

#Esta función resultó estar poco optimizada pues el tiempo que tomaba para realiar los 
#cálculos era demasiado extenso debido a que daba un resultado simbólico y numérico

def pol_char_from_lap(mat,n):
    
   
    simm_mat=mat_shape(mat,n)
    
    Adj_to_nx=nx.from_numpy_matrix(simm_mat)
    L_from_Adj=nx.laplacian_matrix(Adj_to_nx)
    
    L_arr=L_from_Adj.toarray()
    L_np=np.matrix(L_arr)
    
    deg_Mat=L_np+simm_mat
    
    L_WO_Sign=deg_Mat+simm_mat
    
    data_modx2=sp.Matrix(L_WO_Sign)
    
    lamda=symbols("lamda")
    pol=data_modx2.charpoly(lamda)


    
    return print(pol),print(pol.coeffs())

## Análisis de la matriz laplciana sin signo para datos UNS

### 1000 genes

In [16]:
pol_char_from_signless_lap(A,1000)

array([ 1.00000000e+00, -4.11320000e+05,  8.45009745e+10, ...,
                   inf,            -inf,             inf])

## 2000 genes

In [17]:
pol_char_from_signless_lap(A,2000)

array([ 1.00000000e+00, -1.64580800e+06,  1.35361396e+12, ...,
                   inf,            -inf,             inf])

## 3000 genes 

In [19]:
inicio = time.perf_counter()

polinomio = pol_char_from_signless_lap(A,3000)

final = time.perf_counter()

print("Los coeficientes del polinomiocaracterístico son", polinomio)

print("El tiempo de ejecución fue de ", final-inicio,"segundos")


Los coeficientes del polinomiocaracterístico son [ 1.00000000e+00 -3.69525200e+06  6.82499985e+12 ...             inf
            -inf             inf]
El tiempo de ejecución fue de  44.23430857100038 segundos


## 4000 genes

In [20]:
inicio = time.perf_counter()

polinomio = pol_char_from_signless_lap(A,4000)

final = time.perf_counter()

print("Los coeficientes del polinomio característico son", polinomio)

print("El tiempo de ejecución fue de ", final-inicio,"segundos")

Los coeficientes del polinomiocaracterístico son [ 1.00000000e+00 -6.56978600e+06  2.15752514e+13 ...             inf
            -inf             inf]
El tiempo de ejecución fue de  107.77007454400018 segundos


## 5000 genes

In [21]:
inicio = time.perf_counter()

polinomio = pol_char_from_signless_lap(A,5000)

final = time.perf_counter()

print("Los coeficientes del polinomio característico son", polinomio)

print("El tiempo de ejecución fue de ", final-inicio,"segundos")

Los coeficientes del polinomio característico son [ 1.0000000e+00 -1.0210350e+07  5.2114444e+13 ...            inf
           -inf            inf]
El tiempo de ejecución fue de  205.6521276049998 segundos


## 6000 genes


In [22]:
inicio = time.perf_counter()

polinomio = pol_char_from_signless_lap(A,6000)

final = time.perf_counter()

print("Los coeficientes del polinomio característico son", polinomio)

print("El tiempo de ejecución fue de ", final-inicio,"segundos")

Los coeficientes del polinomio característico son [ 1.00000000e+00 -1.46468160e+07  1.07245459e+14 ...             inf
            -inf             inf]
El tiempo de ejecución fue de  436.0252981089998 segundos


## 7000 genes

In [23]:
inicio = time.perf_counter()

polinomio = pol_char_from_signless_lap(A,7000)

final = time.perf_counter()

print("Los coeficientes del polinomio característico son", polinomio)

print("El tiempo de ejecución fue de ", final-inicio,"segundos")

Los coeficientes del polinomio característico son [ 1.00000000e+00 -1.99368900e+07  1.98709385e+14 ...             inf
            -inf             inf]
El tiempo de ejecución fue de  975.9227132810001 segundos


## 8000 genes

In [24]:
inicio = time.perf_counter()

polinomio = pol_char_from_signless_lap(A,8000)

final = time.perf_counter()

print("Los coeficientes del polinomio característico son", polinomio)

print("El tiempo de ejecución fue de ", final-inicio,"segundos")

Los coeficientes del polinomio característico son [ 1.00000000e+00 -2.60351080e+07  3.38868045e+14 ...             inf
            -inf             inf]
El tiempo de ejecución fue de  1666.6188906359994 segundos


## 9000 genes

In [26]:
inicio = time.perf_counter()

polinomio = pol_char_from_signless_lap(A,9000)

final = time.perf_counter()

print("Los coeficientes del polinomio característico son", polinomio)

print("El tiempo de ejecución fue de ", final-inicio,"segundos")

Los coeficientes del polinomio característico son [ 1.00000000e+00 -3.28173100e+07  5.38423879e+14 ...             inf
            -inf             inf]
El tiempo de ejecución fue de  6737.1762727979985 segundos
